In [1]:
import pandas as pd
from io import StringIO
import requests
import datetime
from etl.utils import *
from etl.import_sql import *
from tw_data.models import *
import time
import json
import numpy as np
import sys,os
sys.path.append("..")
import django
django.setup()
from django.conf import settings
import logging
import datetime

logging.basicConfig(level=logging.INFO)

In [ ]:
class InsiderShareholdingDeclarationTransferCrawlerTW:
    def __init__(self):
        self.target_name = "台股內部人持股轉讓宣告"
        self.sub_market = ["sii", "otc","rotc"]

    def crawl_func(self,market,year,smonth='01',emonth='12'):
        url='https://mops.twse.com.tw/mops/web/ajax_t56sb21'
        form_data={'encodeURIComponent': '1',
        'run': 'Y',
        'step': '1',
        'TYPEK': market,
        'year': year,
        'smonth': smonth,
        'emonth': emonth,
        'sstep': '1',
        'firstin': 'true'}
        res = requests.post(url,data = form_data)
        res.encoding = 'utf-8'
        df = pd.read_html(StringIO(res.text))[0]
        df.columns=[a if a==b else a+'_'+b for a,b in df.columns]
        df=df.astype(str)
        df = df.rename(columns={
         '申報日期': 'date',
         '公司代號': 'stock_id',
         '公司名稱': 'stock_name',
         '申報人身分': 'declarant_identity',
         '姓名': 'name',
         '預定轉讓方式及股數_轉讓方式': 'shares_transfer_method',
         '預定轉讓方式及股數_轉讓股數': 'transferred_shares_num',
         '每日於盤中交易最大得轉讓股數': 'maximum_transferable_shares_in_one_day',
         '受讓人': 'assignee',
         '目前持有股數_自有持股': 'current_shares',
         '目前持有股數_保留運用決定權信託股數': 'current_shares_trust',
         '預定轉讓總股數_自有持股': 'transferred_own_shares_total_num',
         '預定轉讓總股數_保留運用決定權信託股數': 'transferred_trust_shares_total_num',
         '預定轉讓後持股_自有持股': 'after_transfer_own_shareholding',
         '預定轉讓後持股_保留運用決定權信託股數': 'after_transfer_trust_shareholding',
         '是否申報持股未完成轉讓': 'declare_uncompleted_transfer'})

        df['date'] = df['date'].apply(lambda t: year_transfer(t,method='datetime'))
        df['有效轉讓期間']=df['有效轉讓期間'].apply(lambda t:t.replace('nan','000/01/01 ~ 000/01/01'))
        df['start_date']=df['有效轉讓期間'].apply(lambda t: year_transfer(t[:t.index('~')-1],method='datetime'))
        df['end_date']=df['有效轉讓期間'].apply(lambda t: year_transfer(t[t.index('~')+1:],method='datetime'))
        df['declare_uncompleted_transfer']=df['declare_uncompleted_transfer'].apply(lambda t:t.replace('是','1').replace('nan','0'))
        df=df.drop(columns=['異動情形','有效轉讓期間'])
        numeric_col=['transferred_shares_num','maximum_transferable_shares_in_one_day','current_shares','current_shares_trust','transferred_own_shares_total_num','transferred_trust_shares_total_num','after_transfer_own_shareholding','after_transfer_trust_shareholding','declare_uncompleted_transfer']
        df[numeric_col] = df[numeric_col].apply(lambda s: pd.to_numeric(s, errors="coerce"))
        df['market']=market
        df=df.sort_values('date')
        return df


    def crawl_main(self,y_list=None):
        if y_list is None:
            year=datetime.datetime.now().year-1911
            y_list=[str(year)]
        try:
            data=[]
            for m in self.sub_market:
                for y in y_list:
                    try:
                        df=self.crawl_func(m,y)
                    except Exception as e:   
                        logger.error(m,e)
                        return None
                    data.append(df)
                    time.sleep(10)
            result=pd.concat(data).sort_values('date')
        except Exception as e:
            logger.error(e)
            return None
        return result


In [ ]:
# one market 
df=InsiderShareholdingDeclarationTransferCrawlerTW().crawl_func(market='rotc',year='102',smonth='01',emonth='12')
df

In [ ]:
# all market
df=InsiderShareholdingDeclarationTransferCrawlerTW().crawl_main(y_list=['103'])
df

# generate sql code

In [ ]:
tail='  PRIMARY KEY (`id`),\n  UNIQUE INDEX `unique_idx` (`stock_id` ASC, `date` ASC) VISIBLE,\n  INDEX `stock_id_idx` (`stock_id` ASC) VISIBLE,\n  INDEX `date_idx` (`date` ASC) VISIBLE,\n  INDEX `name_idx` (`name` ASC) VISIBLE);'
SqlCreateTable(ss).generate_table('tw_data','insider_shareholding_declaration_transfer',tail)


# import data

In [ ]:
y_list=[str(i) for i in range(103,110)]
y_list

In [ ]:
c=InsiderShareholdingDeclarationTransferCrawlerTW()
# SqlImporter.add_to_sql(InsiderShareholdingDeclarationTransfer, c.crawl_main(y_list=y_list))
SqlImporter.add_to_sql(InsiderShareholdingDeclarationTransfer, c.crawl_main())

In [ ]:
c=InsiderShareholdingDeclarationTransferCrawlerTW()
SqlImporter.add_to_sql(InsiderShareholdingDeclarationTransfer, df)

In [2]:
from etl.tasks import crawler_task_once
crawler_task_once(
'InsiderShareholdingDeclarationTransferCrawlerTW'  ,'crawl_main'  , 'InsiderShareholdingDeclarationTransfer' )

INFO:etl.import_sql:Finish!<class 'tw_data.models.InsiderShareholdingDeclarationTransfer'> date:from 2020-01-02 to 2020-11-20 bulk_create:0
INFO:etl.import_sql:Finish!<class 'tw_data.models.InsiderShareholdingDeclarationTransfer'> date:from 2020-01-02 to 2020-11-20 bulk_update:1861


"Finish!<class 'tw_data.models.InsiderShareholdingDeclarationTransfer'> date:from 2020-01-02 to 2020-11-20 bulk_create:0,Finish!<class 'tw_data.models.InsiderShareholdingDeclarationTransfer'> date:from 2020-01-02 to 2020-11-20 bulk_update:1861"